In [2]:
import math
from random import random
import numpy as np

In [71]:
def print_mean_var(seq, mean, var, x, y):
    print("mean real: " + str(np.array(seq).mean()))
    print("mean theory: " + str(mean(x, y)))
    print("mean difference: " + str(np.array(seq).mean() - mean(x, y)))
    print()
    print("variance real: " + str(np.array(seq).var()))
    print("variance theory: " + str(var(x, y)))
    print("variance difference: " + str(np.array(seq).var() - var(x, y)))



def get_next_gauss(N):
    r = random()
    return math.sqrt(12 / N) * (sum([random() for _ in range(N)]) - N / 2)


def gauss(m, s, N, n):
    for _ in range(n):
        yield m + s * get_next_gauss(N)
    

def mean_gauss_theory(m, s):
    return m


def variance_gauss_theory(m, s):
    return s ** 2

def variance(seq):
    sum = 0
    mean = np.array(seq).mean()

    for el in seq:
        sum += (el - mean) ** 2
    return  1 / (len(seq) - 1) * sum

In [47]:
N = 48
n = 10000
m = -4
s = 2

gauss_seq = list(gauss(m, s, N, n))
print_mean_var(gauss_seq, mean_gauss_theory, variance_gauss_theory, m, s)


mean real: -4.015168368268503
mean theory: -4
mean difference: -0.015168368268502874

variance real: 3.900923358705383
variance theory: 4
variance difference: -0.09907664129461713


In [48]:
a = 0.5

def mean_exp_theory(a, *args):
    return 1.0 / a

def variance_exp_theory(a, *args):
    return 1 / (a ** 2)

def exponential(a, n):
    for _ in range(n):
        yield -1 / a * math.log(random())
        
exp_seq = list(exponential(a, n))
print((np.array(exp_seq)).mean())
print_mean_var(exp_seq, mean_exp_theory, variance_exp_theory, a, None)

2.0117420863110556
mean real: 2.0117420863110556
mean theory: 2.0
mean difference: 0.011742086311055555

variance real: 3.949112124265558
variance theory: 4.0
variance difference: -0.0508878757344422


In [49]:
a = 0
b = 1.5

def mean_log_theory(a, b):
    return a

def variance_log_theory(a, b):
    return (math.pi ** 2) / 3 * b ** 2

def logistic(a, b, n):
    for _ in range(n):
        r = random()
        yield a + b * math.log(r  /(1 - r))
        
log_seq = list(logistic(a, b, n))
print_mean_var(log_seq, mean_log_theory, variance_log_theory, a, b)

mean real: 0.03104303574324872
mean theory: 0
mean difference: 0.03104303574324872

variance real: 7.6472777923197155
variance theory: 7.4022033008170185
variance difference: 0.245074491502697


In [50]:
pi = 0.7

3) (1 балл) Осуществить моделирование n = 10000 реализаций случайной величины из стандартного нормального закона распределения N(0, 1), используя преобразование Бокса — Мюллера http://ru.wikipedia.org/wiki/Преобразование_Бокса_—_Мюллера (в источнике  Лобач В.И. [и др] такой метод моделирования называется: моделирование, «используя функциональное преобразование»).


In [65]:
def box_muller(n):
    for i in range(n):
        x = random() * 2 - 1
        y = random() * 2 - 1
        s = x ** 2 + y ** 2
    
        while s > 1 or s == 0:
            x = random() * 2 - 1
            y = random() * 2 - 1
            s = x ** 2 + y ** 2
    
        yield x *( -2 * math.log(s) / s) ** 0.5
        yield y *( -2 * math.log(s) / s) ** 0.5

bm_seq = list(box_muller(n))
for i in range(20):
    print(bm_seq[i])

1.119023236102204
0.1644865169408829
-0.5060345197485131
-0.8953188938538177
-0.471049953623506
-0.49036588352963056
-0.9245028087414007
0.4769465805735644
-0.944218037158646
-0.12471060065791993
0.25271105709777747
0.03341529260049195
-1.1100559939047274
0.7850655143883184
-0.13686537145194197
1.5436312570321271
0.3197186460637468
-1.3691845203959083
1.079384047971998
0.49028525982253884


4) (1 балл) для смоделированной в бонусном пункте 3 выборки оценить коэффициент корреляции между элементами, стоящими на четных позициях, и элементах, стоящих на нечетных позициях.

In [76]:
odd_elements = [bm_seq[i] for i in range(0, n, 2)]
even_elements = [bm_seq[i] for i in range(1, n, 2)]

def correlation(seq1, seq2):
    mean1 = np.array(seq1).mean()
    mean2 = np.array(seq2).mean()
    var1 = variance(seq1)
    var2 = variance(seq2)
    
    corr = 0.
    for i in range(len(seq1)):
        corr += (seq1[i] - mean1) * (seq2[i] - mean2)

    return corr / (var1 * var2) ** 0.5 / len(seq1)
correlation(odd_elements, even_elements)

-0.005465477203856746

-0.005465477203856746